searching + cleaning + fetching 

In [4]:
import os
import json
import time
import requests
import pandas as pd
from sqlalchemy import create_engine, text

# ---------------------------
# Config
# ---------------------------
TECH_TERMS = [
    "lithium-ion battery",
    "fuel cell",
    "hydrogen storage",
    "solar photovoltaic",
    "wind turbine",
    "3D printing",
    "computer vision",
    "wireless charging",
    "radar sensor",
    "autonomous driving",
]

# Tail truncation (because of delays / incomplete years)
PATENT_TAIL_TRUNC = 3   # drop last 3 years of patent data
PUB_TAIL_TRUNC    = 1   # drop last 1 year of publication data

MAX_RESULTS_PATENTS = 600
REQUEST_TIMEOUT = 120

# DB connection – expects DATABASE_URL in env (same as the Flask app)
DATABASE_URL = os.getenv("DATABASE_URL")
if not DATABASE_URL:
    raise RuntimeError("DATABASE_URL not set. Put it in .env to match the backend (Postgres).")

engine = create_engine(DATABASE_URL)

# ---------------------------
# Helpers
# ---------------------------

def load_backend_port():
    """
    Try several locations; user said '/backend_prot.txt'.
    The app writes to 'frontend/public/backend_port.txt' and 'shared_path.txt' too.
    """
    candidates = [
        "/backend_prot.txt",
        "backend_prot.txt",
        "shared_path.txt",
        os.path.join("frontend", "public", "backend_port.txt"),
        os.getenv("NEXT_PUBLIC_BACKEND_PORT", "").strip() or None
    ]
    for c in candidates:
        if not c:
            continue
        if os.path.isfile(c):
            try:
                return int(open(c, "r").read().strip())
            except Exception:
                pass
        # If it's already a number-like string from env:
        try:
            return int(c)
        except Exception:
            pass
    raise RuntimeError("Could not determine backend port from any known path/env.")

def patents_query_payload(term: str) -> dict:
    """
    Build the JSON for /api/search_ops (OPS-based).
    Uses a simple single-keyword group on 'title' to keep it robust.
    """
    return {
        "query": {
            "group1": {
                "type": "group",
                "operator": "AND",
                "keywords": [
                    {"type": "keyword", "word": term, "rule_op": "any", "field": "title"}
                ]
            }
        },
        "max_results": MAX_RESULTS_PATENTS
    }

def post_json(url: str, payload: dict):
    r = requests.post(url, json=payload, timeout=REQUEST_TIMEOUT)
    r.raise_for_status()
    return r.json()

def get_publications_by_year() -> pd.DataFrame:
    """
    Query the research_data3 table directly and aggregate by year.
    """
    sql = """
        SELECT year::int AS year, COUNT(*)::int AS count
        FROM research_data3
        WHERE year IS NOT NULL
        GROUP BY year
        ORDER BY year
    """
    return pd.read_sql(sql, engine)

def get_patents_by_first_filing_year() -> pd.DataFrame:
    """
    Query raw_patents first_filing_year and aggregate.
    (Same idea as your /api/patents/first_filing_years route.)
    """
    sql = """
        SELECT first_filing_year::int AS year, COUNT(*)::int AS count
        FROM raw_patents
        WHERE first_filing_year IS NOT NULL
        GROUP BY first_filing_year
        ORDER BY year
    """
    return pd.read_sql(sql, engine)

def truncate_tail_years(df: pd.DataFrame, tail: int, year_col="year"):
    if df.empty or tail <= 0:
        return df.copy()
    max_year = int(df[year_col].max())
    cutoff = max_year - tail  # keep <= cutoff
    return df[df[year_col] <= cutoff].copy()

def fill_missing_years(df: pd.DataFrame, year_col="year", count_col="count"):
    if df.empty:
        return df
    years = pd.Series(sorted(df[year_col].unique()))
    yr_min, yr_max = int(years.min()), int(years.max())
    full = pd.DataFrame({year_col: list(range(yr_min, yr_max + 1))})
    out = full.merge(df[[year_col, count_col]], on=year_col, how="left").fillna({count_col: 0})
    out[count_col] = out[count_col].astype(int)
    return out

def best_pub_to_patent_lag(pub_series: pd.Series, pat_series: pd.Series, max_lag=5):
    """
    Simple cross-correlation (same-year alignment baseline).
    Returns (best_lag, corr_at_best_lag). Positive lag means patents lag pubs.
    """
    # Align on common years
    idx = pub_series.index.intersection(pat_series.index)
    p = pub_series.loc[idx].astype(float)
    a = pat_series.loc[idx].astype(float)
    if len(p) < 3:
        return 0, float("nan")
    best = (0, -1.0)  # (lag, corr)
    for lag in range(0, max_lag + 1):
        # Shift patents backward by 'lag' to compare pubs[t] vs patents[t+lag]
        aligned = a.shift(-lag)
        s = pd.concat([p, aligned], axis=1).dropna()
        if len(s) < 3:
            continue
        corr = s.iloc[:, 0].corr(s.iloc[:, 1])
        if corr is not None and corr > best[1]:
            best = (lag, float(corr))
    return best

# ---------------------------
# Main collection procedure
# ---------------------------

def collect_series(tech_terms=TECH_TERMS,
                   patent_tail=PATENT_TAIL_TRUNC,
                   pub_tail=PUB_TAIL_TRUNC):
    port = load_backend_port()
    base = f"http://localhost:{port}"
    pubs_endpoint    = f"{base}/api/scientific_search_merge"
    patents_endpoint = f"{base}/api/search_ops"

    all_pubs_rows = []
    all_pat_rows  = []
    lag_rows      = []

    for term in tech_terms:
        print(f"\n=== Collecting for tech='{term}' ===")

        # --- Publications: call endpoint, then aggregate from DB
        try:
            print("Publications: calling /api/scientific_search_merge …")
            _ = post_json(pubs_endpoint, {"query": term})
            # Immediately query DB (table gets overwritten per term)
            pub_df = get_publications_by_year()
            pub_df = truncate_tail_years(pub_df, pub_tail, "year")
            pub_df = fill_missing_years(pub_df, "year", "count")
            pub_df["tech"] = term
            pub_df = pub_df[["tech", "year", "count"]].rename(columns={"count": "pub_count"})
            all_pubs_rows.append(pub_df)
            print(f"  -> pubs years {pub_df['year'].min()}–{pub_df['year'].max()} (n={len(pub_df)})")
        except Exception as e:
            print(f"  !! publications failed for '{term}': {e}")

        # --- Patents: call endpoint, then aggregate from DB
        try:
            print("Patents: calling /api/search_ops …")
            payload = patents_query_payload(term)
            _ = post_json(patents_endpoint, payload)
            pat_df = get_patents_by_first_filing_year()
            pat_df = truncate_tail_years(pat_df, patent_tail, "year")
            pat_df = fill_missing_years(pat_df, "year", "count")
            pat_df["tech"] = term
            pat_df = pat_df[["tech", "year", "count"]].rename(columns={"count": "patent_count"})
            all_pat_rows.append(pat_df)
            print(f"  -> pats years {pat_df['year'].min()}–{pat_df['year'].max()} (n={len(pat_df)})")
        except Exception as e:
            print(f"  !! patents failed for '{term}': {e}")

        # Optional: xcorr-based lag detection for this tech (uses overlapping years)
        try:
            if len(all_pubs_rows) and len(all_pat_rows):
                pub_cur = all_pubs_rows[-1].set_index("year")["pub_count"]
                pat_cur = all_pat_rows[-1].set_index("year")["patent_count"]
                lag, corr = best_pub_to_patent_lag(pub_cur, pat_cur, max_lag=5)
                lag_rows.append({"tech": term, "best_lag_years": int(lag), "xcorr": corr})
                print(f"  -> best pubs→patents lag: {lag}y (xcorr={corr:.3f})")
        except Exception as e:
            print(f"  !! lag calc failed for '{term}': {e}")

    pubs_df    = pd.concat(all_pubs_rows, ignore_index=True) if all_pubs_rows else pd.DataFrame(columns=["tech","year","pub_count"])
    patents_df = pd.concat(all_pat_rows,  ignore_index=True) if all_pat_rows else pd.DataFrame(columns=["tech","year","patent_count"])
    lag_df     = pd.DataFrame(lag_rows) if lag_rows else pd.DataFrame(columns=["tech","best_lag_years","xcorr"])

    # Ensure integer year
    for df in (pubs_df, patents_df):
        if "year" in df.columns:
            df["year"] = df["year"].astype(int)

    return pubs_df, patents_df, lag_df


if __name__ == "__main__":
    pubs_df, patents_df, lag_df = collect_series()

    # Save artifacts (optional)
    pubs_df.to_csv("pubs_df_collected.csv", index=False)
    patents_df.to_csv("patents_df_collected.csv", index=False)
    lag_df.to_csv("pubs_to_patents_best_lag.csv", index=False)

    print("\nDone. Samples:")
    print(pubs_df.head())
    print(patents_df.head())
    print(lag_df.head())


RuntimeError: DATABASE_URL not set. Put it in .env to match the backend (Postgres).

#utilities 

In [ ]:
import numpy as np
import pandas as pd

# ---------- utilities

def _to_year_end(year_series: pd.Series) -> pd.Series:
    # Ensure Dec-31 ds per year
    return pd.PeriodIndex(year_series.astype(int), freq='Y').to_timestamp(how='end')

def _align_and_corr(pat_s: pd.Series, pub_s: pd.Series, lag: int, detrend: str) -> float:
    """
    Returns Pearson correlation between patents_t and publications_{t-lag},
    after optional detrending. Returns np.nan if not enough overlap.
    """
    df = pd.DataFrame({'pat': pat_s, 'pub': pub_s}).copy()
    df['pub'] = df['pub'].shift(lag)
    df = df.dropna()

    if len(df) < 3:  # too few points for safe corr
        return np.nan

    if detrend == 'diff':
        df = df.diff().dropna()
    elif detrend == 'pct':
        df = df.pct_change().replace([np.inf, -np.inf], np.nan).dropna()
    elif detrend == 'zscore':
        df = (df - df.mean()) / df.std(ddof=0)

    if len(df) < 3:
        return np.nan

    return df['pat'].corr(df['pub'])

def infer_publication_lag_per_tech(
    pubs_df: pd.DataFrame,
    patents_df: pd.DataFrame,
    max_lag: int = 5,
    detrend: str = 'diff',
    min_overlap: int = 8,
    prefer_nonnegative: bool = True
) -> pd.DataFrame:
    """
    For each tech, pick the lag in [0..max_lag] maximizing Pearson corr between
    patents_t and publications_{t-lag} (after detrending). Returns a DataFrame:
    [tech, best_lag, best_corr, overlap_years].
    """
    results = []
    techs = sorted(set(pubs_df['tech']).intersection(set(patents_df['tech'])))

    for tech in techs:
        pat = patents_df.loc[patents_df['tech'] == tech, ['year', 'patent_count']].set_index('year')['patent_count']
        pub = pubs_df.loc[pubs_df['tech'] == tech, ['year', 'pub_count']].set_index('year')['pub_count']

        # Restrict to overlapping years generously (we’ll shift inside)
        yrs = sorted(set(pat.index).intersection(set(pub.index)))
        if len(yrs) < min_overlap:
            # Not enough to infer reliably; default lag=1
            results.append({'tech': tech, 'best_lag': 1, 'best_corr': np.nan, 'overlap_years': len(yrs)})
            continue

        best = {'lag': 0, 'corr': -np.inf}
        for lag in range(0, max_lag + 1):
            # build aligned frame to count overlap for this lag
            df = pd.DataFrame({'pat': pat, 'pub': pub.shift(lag)}).dropna()
            overlap = len(df)
            if overlap < min_overlap:
                continue
            r = _align_and_corr(pat, pub, lag=lag, detrend=detrend)
            if np.isnan(r):
                continue
            if prefer_nonnegative and r < 0:
                continue
            if r > best['corr']:
                best = {'lag': lag, 'corr': r, 'overlap': overlap}

        if best['corr'] == -np.inf:
            # fallback if nothing met criteria
            results.append({'tech': tech, 'best_lag': 1, 'best_corr': np.nan, 'overlap_years': len(yrs)})
        else:
            results.append({'tech': tech, 'best_lag': best['lag'], 'best_corr': best['corr'], 'overlap_years': best['overlap']})

    return pd.DataFrame(results)


#integrate the lag into prophet prep 

In [ ]:
def add_pub_regressor_column(
    patents_df: pd.DataFrame,
    pubs_df: pd.DataFrame,
    lag_map: dict
) -> pd.DataFrame:
    """
    Merges a lagged publication regressor named 'pub_reg' into the patents_df per tech.
    Expects patents_df: [tech, year, patent_count]
            pubs_df:    [tech, year, pub_count]
    lag_map: {tech: best_lag}
    Returns a tidy DF with columns: tech, year, patent_count, pub_reg
    """
    out = []
    for tech, g_pat in patents_df.groupby('tech'):
        lag = int(lag_map.get(tech, 1))
        g_pub = pubs_df[pubs_df['tech'] == tech].copy()
        g_pub = g_pub.assign(year=lambda d: d['year'].astype(int) + lag)  # shift forward so that at year t, pub_reg = pubs_{t-lag}
        g_pub = g_pub.rename(columns={'pub_count': 'pub_reg'})[['year', 'pub_reg']]

        m = g_pat.merge(g_pub, on='year', how='left')
        out.append(m.assign(tech=tech))
    return pd.concat(out, ignore_index=True)


during training & future construction 

In [ ]:
from prophet import Prophet

def _prep_prophet_df_for_patents(df_tech: pd.DataFrame) -> pd.DataFrame:
    # df_tech: [year, patent_count, pub_reg]
    df = df_tech.copy()
    df['ds'] = _to_year_end(df['year'])
    df = df.rename(columns={'patent_count': 'y'})
    # Prophet requires regressor present in both history and future
    return df[['ds', 'y', 'pub_reg']].dropna(subset=['y', 'pub_reg'])  # ensure no NaNs

def forecast_patents_with_regressor_auto_lag(
    patents_df: pd.DataFrame,
    pubs_df: pd.DataFrame,
    pub_forecasts: dict | None,
    horizon: int,
    lag_table: pd.DataFrame
) -> dict:
    """
    per-tech Prophet using auto-selected lag. If `pub_forecasts` is None, caller must pass a
    pubs scenario df per tech in same schema as pubs_df (years extended).
    Returns: dict[tech] -> forecast DataFrame with columns ['ds','yhat','yhat_lower','yhat_upper']
    """
    lag_map = dict(zip(lag_table['tech'], lag_table['best_lag']))
    # Build training table with 'pub_reg'
    train_tbl = add_pub_regressor_column(patents_df, pubs_df, lag_map)
    forecasts = {}

    for tech, g in train_tbl.groupby('tech'):
        dfp = _prep_prophet_df_for_patents(g[['year', 'patent_count', 'pub_reg']])
        if len(dfp) < 5:
            continue  # not enough data to fit

        m = Prophet(yearly_seasonality=False, daily_seasonality=False)
        m.add_regressor('pub_reg')
        m.fit(dfp)

        # build future years
        last_year = int(g['year'].max())
        future_years = [last_year + i for i in range(1, horizon+1)]
        future = pd.DataFrame({'year': future_years})

        # build future pub_reg: use pub forecasts (or scenario) then shift by lag
        lag = lag_map.get(tech, 1)
        if pub_forecasts is not None and tech in pub_forecasts:
            fut_pub = pub_forecasts[tech].copy()  # expected columns: ['year','pub_count']
        else:
            raise ValueError("Provide pub_forecasts per tech or implement scenario generator.")

        fut_pub_shifted = fut_pub.copy()
        fut_pub_shifted['year'] = fut_pub_shifted['year'].astype(int) + lag
        fut_pub_shifted = fut_pub_shifted.rename(columns={'pub_count': 'pub_reg'})[['year','pub_reg']]

        future = future.merge(fut_pub_shifted, on='year', how='left')
        future['ds'] = _to_year_end(future['year'])
        future = future[['ds', 'pub_reg']]

        # concatenate with history to let Prophet make future df
        # Prophet needs the regressor in 'future' frame we pass to predict
        fcst = m.predict(future)
        forecasts[tech] = fcst[['ds','yhat','yhat_lower','yhat_upper']].copy()

    return forecasts


publications forecast + scenario hooks (unchanged , but shown for completeness)

In [ ]:
def forecast_publications_per_tech(pubs_df: pd.DataFrame, horizon: int) -> dict:
    out = {}
    for tech, g in pubs_df.groupby('tech'):
        df = g[['year','pub_count']].copy()
        df['ds'] = _to_year_end(df['year'])
        df = df.rename(columns={'pub_count':'y'})
        m = Prophet(yearly_seasonality=False, daily_seasonality=False)
        m.fit(df[['ds','y']])
        future = m.make_future_dataframe(periods=horizon, freq='Y')
        fcst = m.predict(future)
        # Extract future years as integer for downstream merging
        tmp = fcst[['ds','yhat']].copy()
        tmp['year'] = tmp['ds'].dt.year
        out[tech] = tmp.rename(columns={'yhat':'pub_count'})[['year','pub_count']]
    return out

def generate_pub_scenario(pub_hist: pd.DataFrame, horizon: int, method='flat'):
    last_year = int(pub_hist['year'].max())
    hist = pub_hist.sort_values('year')
    if method == 'flat':
        val = hist['pub_count'].iloc[-1]
        fut = [val]*horizon
    elif method == 'linear_trend':
        if len(hist) >= 6:
            slope = (hist['pub_count'].iloc[-1] - hist['pub_count'].iloc[-6]) / 5.0
        else:
            slope = 0.0
        start = hist['pub_count'].iloc[-1]
        fut = [start + slope*(i+1) for i in range(horizon)]
    years = [last_year + i for i in range(1, horizon+1)]
    return pd.DataFrame({'year': years, 'pub_count': fut})


evaluation sketch (showing where auto-lag plugs in )

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate_prophet_models_with_auto_lag(
    pubs_df: pd.DataFrame,
    patents_df: pd.DataFrame,
    horizon: int = 5,
    test_years: int = 5,
    max_lag: int = 5,
    pub_trunc: int = 1,
    pat_trunc: int = 3
) -> pd.DataFrame:
    # assume truncation already applied during build; if not, apply here

    # split train/test by year per tech AFTER truncation
    metrics = []
    for tech in sorted(set(pubs_df['tech']).intersection(patents_df['tech'])):
        g_pub = pubs_df[pubs_df['tech']==tech].copy()
        g_pat = patents_df[patents_df['tech']==tech].copy()
        cutoff = int(min(g_pub['year'].max(), g_pat['year'].max()) - test_years)
        pub_train, pub_test = g_pub[g_pub['year']<=cutoff], g_pub[g_pub['year']>cutoff]
        pat_train, pat_test = g_pat[g_pat['year']<=cutoff], g_pat[g_pat['year']>cutoff]

        # 1) publications forecast (train only)
        pub_fcsts = forecast_publications_per_tech(pub_train, horizon=len(pub_test))
        pub_pred = pub_fcsts.get(tech, pd.DataFrame(columns=['year','pub_count']))
        pub_pred = pub_pred[pub_pred['year'].isin(pub_test['year'])]
        if len(pub_pred):
            mae_pub = mean_absolute_error(pub_test['pub_count'].values, pub_pred['pub_count'].values)
            rmse_pub = np.sqrt(mean_squared_error(pub_test['pub_count'].values, pub_pred['pub_count'].values))
        else:
            mae_pub = rmse_pub = np.nan

        # 2) auto-lag inference on TRAIN ONLY
        lag_tab = infer_publication_lag_per_tech(pub_train, pat_train, max_lag=max_lag)
        lag_map = dict(zip(lag_tab['tech'], lag_tab['best_lag']))

        # 3) patents baseline (no regressor)
        # fit simple Prophet on patents only (train)
        dfp = pat_train[['year','patent_count']].copy()
        dfp['ds'] = _to_year_end(dfp['year'])
        dfp = dfp.rename(columns={'patent_count':'y'})
        m_base = Prophet(yearly_seasonality=False, daily_seasonality=False)
        m_base.fit(dfp[['ds','y']])
        fut = pd.DataFrame({'ds': _to_year_end(pat_test['year'])})
        base_pred = m_base.predict(fut)
        base_yhat = base_pred['yhat'].values
        mae_base = mean_absolute_error(pat_test['patent_count'].values, base_yhat)
        rmse_base = np.sqrt(mean_squared_error(pat_test['patent_count'].values, base_yhat))

        # 4) patents with regressor (auto-lag)
        # build regressor on TRAIN, use pub forecast (from step 1) for TEST → then shift
        # construct per-tech pubs future by concatenating pub_train with pub_pred for alignment
        pub_future = pd.concat([pub_train[['year','pub_count']], pub_pred[['year','pub_count']]], ignore_index=True)
        lagged_train = add_pub_regressor_column(pat_train, pub_train, lag_map)
        df_train = _prep_prophet_df_for_patents(lagged_train[['year','patent_count','pub_reg']])

        m = Prophet(yearly_seasonality=False, daily_seasonality=False)
        m.add_regressor('pub_reg')
        m.fit(df_train)

        lag = lag_map.get(tech, 1)
        pub_future_shift = pub_future.copy()
        pub_future_shift['year'] = pub_future_shift['year'].astype(int) + lag
        pub_future_shift = pub_future_shift.rename(columns={'pub_count':'pub_reg'})[['year','pub_reg']]

        fut_pat = pd.DataFrame({'year': pat_test['year']})
        fut_pat = fut_pat.merge(pub_future_shift, on='year', how='left')
        fut_pat['ds'] = _to_year_end(fut_pat['year'])
        fc = m.predict(fut_pat[['ds','pub_reg']])
        mae_reg = mean_absolute_error(pat_test['patent_count'].values, fc['yhat'].values)
        rmse_reg = np.sqrt(mean_squared_error(pat_test['patent_count'].values, fc['yhat'].values))

        metrics.append({
            'tech': tech,
            'best_lag': lag,
            'mae_pubs': mae_pub, 'rmse_pubs': rmse_pub,
            'mae_patents_with_reg': mae_reg, 'rmse_patents_with_reg': rmse_reg,
            'mae_patents_baseline': mae_base, 'rmse_patents_baseline': rmse_base
        })

    return pd.DataFrame(metrics)
